Thanks to: https://www.kaggle.com/drcapa/happywhale-2022-starter



And the preprocessor step is based on an old competition but I don't have notebook link anymore

# Libraries

In [ ]:
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path

In [ ]:
path = '/kaggle/input/happy-whale-and-dolphin/'
os.listdir(path)

# Load Data

In [ ]:
train_data = pd.read_csv(path+'train.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

# Overview

In [ ]:
print('Number train samples:', len(train_data))
print('Number train images:', len(os.listdir(path+'train_images/')))
print('Number test images:', len(os.listdir(path+'test_images/')))

In [ ]:
train_data.head()

# EDA

In [ ]:
train_data['species'].value_counts()

# Load Single Image
We plot the first image of of the train data.

In [ ]:
row = 0
file = train_data.loc[row, 'image']
species = train_data.loc[row, 'species']

img = cv2.imread(path+'train_images/'+file)
print('Shape:', img.shape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_title(species)
plt.show()

# Plot Examples
We plot example images of the breed top 10.

In [ ]:
def plot_examples(category = 'bottlenose_dolphin'):
    """ Plot 5 images of a given category """
    
    fig, axs = plt.subplots(1, 5, figsize=(25, 20))
    fig.subplots_adjust(hspace = .1, wspace=.1)
    axs = axs.ravel()
    temp = train_data[train_data['species']==category].copy()
    temp.index = range(len(temp.index))
    for i in range(5):
        file = temp.loc[i, 'image']
        species = temp.loc[i, 'species']
        img = cv2.imread(path+'train_images/'+file)
        axs[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[i].set_title(species)
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
    plt.show()

In [ ]:
plot_examples(category = 'bottlenose_dolphin')

In [ ]:
plot_examples(category = 'beluga')

In [ ]:
plot_examples(category = 'humpback_whale')

# Image Preprocessing
As we can see the images have different format: landscape or portrait. For the neural network we need a standard size. So we have to prepare the data. 

In [ ]:

def crop_image_from_gray(img, tol=7):
    """
    Applies masks to the orignal image and 
    returns the a preprocessed image with 
    3 channels
    
    :param img: A NumPy Array that will be cropped
    :param tol: The tolerance used for masking
    
    :return: A NumPy array containing the cropped image
    """
    # If for some reason we only have two channels
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    # If we have a normal RGB images
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img
    
def preprocess_image(image, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    
    :param img: A NumPy Array that will be cropped
    :param sigmaX: Value used for add GaussianBlur to the image
    
    :return: A NumPy array containing the preprocessed image
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (224, 224))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image


In [ ]:
image_size = 128

# Data Generator
We define a data generator to load the data on demand.

**Coming soon**

In [ ]:
y_train = train_data['species']
y_train = pd.get_dummies(y_train)

# Model
**Coming soon**

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
base_model = EfficientNetB0(include_top=False, weights='imagenet' , input_shape=(224,224,3))
def add_new_last_layer(base_model, nb_classes):
    #x = base_model.output
    #x = GlobalAveragePooling2D()(x)
    #x = Dense(512, activation='relu')(x)
    #predictions = Dense(nb_classes, activation='softmax')(x)
    #model = Model(input=base_model.input, output=predictions)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(5, activation='relu')(x)
    final_output = Dense(nb_classes, activation='softmax', name='final_output')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=final_output)
    return model
model = add_new_last_layer(base_model, 15587)
model.save('B0')
model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=360,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.15,
                                   preprocessing_function=preprocess_image, 
                                   rescale=1. / 255)


train_generator = train_datagen.flow_from_dataframe(train_data, 
                                                    x_col='image', 
                                                    y_col='individual_id',
                                                    directory = '/kaggle/input/happy-whale-and-dolphin/train_images/',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical', 
                                                    subset='training')
val_generator = train_datagen.flow_from_dataframe(train_data, 
                                                  x_col='image', 
                                                y_col='individual_id',
                                                  directory = '/kaggle/input/happy-whale-and-dolphin/train_images/',
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical',
                                                  subset='validation')

In [ ]:
hist = model.fit(train_generator, steps_per_epoch=train_generator.samples // 32,
                    epochs=1,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // 32)
#training slowly, needs to improve

# Export

In [ ]:
#next Step